In [1]:
import pandas as pd
from dlpy.transformers.bert_utils import bert_prepare_data
from dlpy.transformers import bert_model
import swat

viya_conn = swat.CAS('10.96.1.152', 5570, 'sasdemo', 'Orion123')


cache_dir = '/home/sasdemo/sentiment_analysis/cache_dir1'
bert = bert_model.BERT_Model(viya_conn,
                             cache_dir,
                             'bert-base-uncased',
                             2,
                             num_hidden_layers=12,
                             max_seq_len=256,
                             verbose=True)

reviews = pd.read_csv(
    '/home/sasdemo/sentiment_analysis/data/Reviews.csv', header=0, encoding='utf-8')
reviews = reviews.head(2000)

t_idx = reviews["Score"] != 3

inputs = reviews[t_idx]["Text"].to_list()
targets = reviews[t_idx]["Score"].to_list()
for ii, val in enumerate(targets):
    inputs[ii] = inputs[ii].replace("<br />", "")
    if (val == 1) or (val == 2):  # negative reviews
        targets[ii] = 1
    elif (val == 4) or (val == 5):  # positive reviews
        targets[ii] = 2


# the arguments max_seq_len and trucation were
num_tgt_var, train, test = bert_prepare_data(viya_conn,
                                             bert.get_tokenizer(),
                                             input_a=inputs,
                                             target=targets,
                                             train_fraction=0.8,
                                             segment_vocab_size=bert.get_segment_size(),
                                             classification_problem=bert.get_problem_type(),
                                             #  truncation=True,
                                             max_seq_len=512,
                                             verbose=True)

bert.compile(num_target_var=num_tgt_var)

bert.load_weights('/home/sasdemo/sentiment_analysis/cache_dir1/bert-base-uncased.kerasmodel.h5',
                  num_target_var=num_tgt_var, freeze_base_model=False)


bert.set_optimizer_parameters(learning_rate=2e-5)
bert.fit(train,
         data_specs=bert.get_data_spec(num_tgt_var),
         optimizer=bert.get_optimizer(),
         text_parms=bert.get_text_parameters(),
         seed=12345)


/opt/anaconda3/lib/python3.7/site-packages/dlpy/transformers/bert_model.py:45: UserWarning: You are using a version of the transformers package (4.0.0) that has not been tested for compatibility.  Unexpected behavior may occur.
  warnings.warn(warn_message,UserWarning)


NOTE: loading base model bert-base-uncased ...
NOTE: base model bert-base-uncased loaded.
NOTE: creating embedding table ... 
NOTE: embedding table created and loaded.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


NOTE: 10% of the observations tokenized.
NOTE: 20% of the observations tokenized.
NOTE: 30% of the observations tokenized.
NOTE: 40% of the observations tokenized.
NOTE: 50% of the observations tokenized.
NOTE: 60% of the observations tokenized.
NOTE: 70% of the observations tokenized.
NOTE: 80% of the observations tokenized.
NOTE: 90% of the observations tokenized.
NOTE: 100% of the observations tokenized.
NOTE: all observations tokenized.

NOTE: uploading training data to table bert_train_data.
NOTE: there are 1444 observations in the training data set.

NOTE: uploading test/validation data to table bert_test_validation_data.
NOTE: there are 394 observations in the test/validation data set.

NOTE: training and test/validation data sets ready.

NOTE: HDF5 file is /home/sasdemo/sentiment_analysis/cache_dir1/bert-base-uncased.kerasmodel.h5
NOTE: Model compiled successfully.
NOTE: Model weights attached successfully!
NOTE: Training based on existing weights.
NOTE:  Synchronous mode is en

,Descr,Value
0,Model Name,classification
1,Model Type,Recurrent Neural Network
2,Number of Layers,92
3,Number of Input Layers,3
4,Number of Output Layers,1
5,Number of Convolutional Layers,0
6,Number of Pooling Layers,0
7,Number of Fully Connected Layers,25
8,Number of Recurrent Layers,1
9,Number of Residual Layers,25


In [3]:
res = bert.predict(test,
                   text_parms=bert.get_text_parameters())
print(res['ScoreInfo'])

                         Descr         Value
0  Number of Observations Read           394
1  Number of Observations Used           394
2  Misclassification Error (%)      5.329949
3                   Loss Error      0.184656
